## 視覺化dataFrame(Table Visualization)
- 注意,在github上,無法顯示顏色
  

## 格式化值,index,column
- 必需install jinja2
- format()
- format_index()


In [19]:
import pandas as pd
import numpy as np
import matplotlib as mpl

df = pd.DataFrame({
    "strings": ["Adam", "Mike"],
    "ints": [1, 3],
    "floats": [1.123, 1000.23]
})
display(df)
s = df.style\
.format(precision=3, thousands=",", decimal=".")\
.format_index(str.upper, axis=1)\
.relabel_index(["row 1", "row 2"], axis=0)
print(type(s))
s 

,strings,ints,floats
0,Adam,1,1.123
1,Mike,3,1000.230


<class 'pandas.io.formats.style.Styler'>


,STRINGS,INTS,FLOATS
row 1,Adam,1,1.123
row 2,Mike,3,"1,000.230"


In [20]:
#綜合操作
weather_df = pd.DataFrame(np.random.rand(10,2)*5,
                        index=pd.date_range(start="2021-01-01", periods=10),
                        columns=["Tokyo", "Taipei"])

display(weather_df)

,Tokyo,Taipei
2021-01-01,3.437691,1.170894
2021-01-02,3.399634,2.147009
2021-01-03,0.501041,0.980159
2021-01-04,0.613549,4.305316
2021-01-05,4.291220,0.547187
2021-01-06,1.861680,3.008590
2021-01-07,4.060481,4.347692
2021-01-08,4.776975,2.947082
2021-01-09,4.125324,1.757245
2021-01-10,1.611601,1.545124


In [21]:
weather_df = pd.DataFrame(np.random.rand(10,2)*5,
                        index=pd.date_range(start="2021-01-01", periods=10),
                        columns=["Tokyo", "Taipei"])

def rain_condition(v):
    if v < 1.75:
        return "Dry"
    elif v < 2.75:
        return "Rain"
    return "Heavy Rain"

def make_pretty(styler):
    styler.set_caption("Weather Conditions")
    styler.format(rain_condition)
    styler.format_index(lambda v: v.strftime("%A"))
    styler.background_gradient(axis=None, vmin=1, vmax=5, cmap="PuBuGn")
    return styler

weather_df.loc["2021-01-04":"2021-01-08"].style.pipe(make_pretty)

,Tokyo,Taipei
Monday,Heavy Rain,Dry
Tuesday,Dry,Heavy Rain
Wednesday,Dry,Heavy Rain
Thursday,Heavy Rain,Heavy Rain
Friday,Dry,Rain


## 隱藏資料
- #### hide()

In [22]:
df = pd.DataFrame(np.random.randn(5,5))
display(df)

df.style \
.hide(subset=[0, 2, 4], axis=0)\
.hide(subset=[0, 2, 4], axis=1)

,0,1,2,3,4
0,0.026931,-0.870425,1.691189,-0.613976,-0.003936
1,-0.910663,1.464458,0.645431,-1.272935,-0.582149
2,-0.553978,-0.219577,1.964844,0.552914,-1.623411
3,1.653594,-1.060446,0.117187,-0.534597,0.226399
4,-0.721739,0.332097,1.143384,0.075472,1.777883


,1,3
1,1.464458,-1.272935
3,-1.060446,-0.534597


In [23]:
show = [0, 2, 4]
df.style\
.hide([row for row in df.index if row not in show], axis=0)\
.hide([col for col in df.columns if col not in show], axis=1)

,0,2,4
0,0.026931,1.691189,-0.003936
2,-0.553978,1.964844,-1.623411
4,-0.721739,1.143384,1.777883


## 連結資料
- concat()

In [27]:
display(df.agg(["sum","mean"]))
summary_style = df.agg(["sum", "mean"]).style\
                                    .format(precision=3)\
                                    .relabel_index(["總分","平均"])
display(summary_style)
df.style.format(precision=3).concat(summary_style)

,0,1,2,3,4
sum,-0.505856,-0.353893,5.562036,-1.793121,-0.205214
mean,-0.101171,-0.070779,1.112407,-0.358624,-0.041043


,0,1,2,3,4
總分,-0.506,-0.354,5.562,-1.793,-0.205
平均,-0.101,-0.071,1.112,-0.359,-0.041


,0,1,2,3,4
0,0.027,-0.870,1.691,-0.614,-0.004
1,-0.911,1.464,0.645,-1.273,-0.582
2,-0.554,-0.220,1.965,0.553,-1.623
3,1.654,-1.060,0.117,-0.535,0.226
4,-0.722,0.332,1.143,0.075,1.778
總分,-0.506,-0.354,5.562,-1.793,-0.205
平均,-0.101,-0.071,1.112,-0.359,-0.041
